# Community detection with a Bayesian stochastic block model (Peixoto, 2017)

Based on the aggregated single-layer graph with edge weights as covariates.

In [82]:
import numpy as np
import igraph as ig
import matplotlib
from graph_tool.all import *

n_per_layer = 33

np.set_printoptions(threshold=np.inf)

with open("outputs/01_dataPreperation/final/CAMaggregated_adj_matrix.csv", "r") as f:
    names = f.readline()

names = names.split('" "')
names[0] = names[0][1:]     # strip the first "
names[-1] = names[-1][:-2]  # strip the last \n

names = dict(enumerate(names))

adj_matrix = np.loadtxt(open("outputs/01_dataPreperation/final/CAMaggregated_adj_matrix.csv", "rb"), delimiter=" ", skiprows=1)


# remove self-loops
np.fill_diagonal(adj_matrix, 0)

G = ig.Graph.Adjacency(matrix=adj_matrix, mode="undirected")
G_edge_list = G.get_edgelist()

# ig.plot(G)
# print(G_edge_list)

G_edge_set = set(G_edge_list)
G_edge_list_new = list(G_edge_set)
G_edge_list_new = sorted(G_edge_list_new, key=lambda x: (x[0], x[1]))
# print(G_edge_list_new)

G_gt = Graph(G_edge_list_new, directed=False)
# G_gt_edge_list = list(G_gt.iter_edges())
# G_gt_edge_list = list(map(lambda x: tuple(x), G_gt_edge_list))
# G_gt_edge_list == G_edge_list
# print(G_gt)

G_gt_weights = G_gt.new_edge_property("int")

for edge in G_edge_list_new:
    G_gt_weights[edge] = G_edge_list.count(edge)

# for edge in G_edge_list_new:
#     print(edge, "has weight", G_gt_weights[edge])

G_gt.edge_properties["weight"] = G_gt_weights


def community_detection():
    np.random.seed(12345)
    graph_tool.seed_rng(12345)

    state = minimize_nested_blockmodel_dl(G_gt, state_args=dict(recs=[G_gt.ep.weight], rec_types=["discrete-binomial"]))

    # improve solution with merge-split
    for i in range(100):
        dS, nattempts, nmoves = state.multiflip_mcmc_sweep(niter=10, beta=np.inf)

    return state


def print_solution(state):
    state.draw(ecmap=(matplotlib.cm.inferno, .6),
            eorder=G_gt.ep.weight, edge_pen_width=prop_to_size(G_gt.ep.weight, 2, 8, power=1),
            edge_gradient=[])

    print("Summary of the hierarchical stochastic block model inferred from the data:")
    print("##############################################################################")
    print("l = level, N = number of nodes, B = number of blocks")
    state.print_summary()

    levels = state.get_levels()

    for i, s in enumerate(levels):
        print("    l", i, ": ", s)
        if s.get_N() == 1:
            num_levels = i
            break


    for j in range(num_levels):
        blocklist = levels[j].get_blocks()
        print("##############################################################################")
        print("LEVEL", j, "with the following blocks with at least 2 members (rest is")
        print("considered background for our purposes):\n")

        for block in set(blocklist):
            indices = [i for i, x in enumerate(blocklist) if x == block]
            if len(indices) > 1:
                print("block", block, "with:", ", ".join(map(lambda x: names[x], indices)))


def test_reproducibility():
    solutions = [[], [], []]

    for i in range(100):
        state = community_detection()

        levels = state.get_levels()

        for i, s in enumerate(levels):
            if s.get_N() == 1:
                num_levels = i
                break

        for j in range(num_levels):
            blocklist = levels[j].get_blocks()

            partitions = []

            for block in set(blocklist):
                indices = [i for i, x in enumerate(blocklist) if x == block]
                partitions += [indices]

            solutions[j] += [partitions]


    for i, level in enumerate(solutions):
        sorted_level = list(sorted(list(map(sorted, level))))
        sorted_level = set(tuple(map(lambda x: tuple(map(tuple, x)), sorted_level)))
        print("level", i, "with", len(sorted_level), "different solutions:")

        for s in sorted_level:
            print(s)

        print("############################################################")

# test_reproducibility()
# ...yields only one solution, which is fine.

state = community_detection()

print_solution(state)

KeyboardInterrupt: 

# Community detection with Ensemble Graph Clustering (ECG) (Poulin & Théberge, 2018)

Based on the aggregated multi-edge graph.
Code from https://codeocean.com/capsule/3898939/tree/v1.

In [10]:
from ecg import *
from random import seed

seed(12345)
ec = G.community_ecg()

ec_partitions = list(ec)
ec_partitions = list(map(lambda x: list(map(lambda y: names[y], x)), ec_partitions))

for i, partition in enumerate(ec_partitions):
    print("partition", i, ":", ",".join(partition))

partition 0 : Akzeptanz eines neuen Materialsystems,enthält Kunststoff,elektronikfrei,autonom,intelligent,umweltschädlich,multifunktional,technologisch
partition 1 : Energie   speichernd,energieeffizient,energieautonom,Energie   generierend
partition 2 : wartungsintensiv,wartungsfrei,selbstheilend,selbstreparierend
partition 3 : aktive Formänderung durch Umwelteinwirkung,reaktionsfähig,passive Formänderung durch Umwelteinwirkung,passive Verhaltensänderung durch Umwelteinwirkung,aktive Verhaltensänderung durch Umwelteinwirkung
partition 4 : ökologisch,nachhaltig,umweltfreundlich
partition 5 : zuverlässig,widerstandsfähig,leicht zerstörbar,haltbar,robust,langlebig
partition 6 : Insekten ähnlich,bioinspiriert,lebensähnlich
